In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys  
import itertools

import tensorflow as tf
import keras

In [7]:
from datasets import load_dataset
dataset = load_dataset("conll2003")

Reusing dataset conll2003 (/Users/rachelung/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63ba56944e35c1943434322a07ceefd79864672041b7834583709af4a5de4664)


In [8]:
df = pd.DataFrame(dataset['train'])
df['sentence_length'] = df.tokens.apply(lambda x: len(x))
df.head()

,chunk_tags,id,ner_tags,pos_tags,tokens,sentence_length
0,"[11, 21, 11, 12, 21, 22, 11, 12, 0]",0,"[3, 0, 7, 0, 0, 0, 7, 0, 0]","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[EU, rejects, German, call, to, boycott, Briti...",9
1,"[11, 12]",1,"[1, 2]","[22, 22]","[Peter, Blackburn]",2
2,"[11, 12]",2,"[5, 0]","[22, 11]","[BRUSSELS, 1996-08-22]",2
3,"[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 1...",3,"[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ...","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 3...","[The, European, Commission, said, on, Thursday...",30
4,"[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 1...",4,"[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ...","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 2...","[Germany, 's, representative, to, the, Europea...",31


In [9]:
words = list(set(itertools.chain.from_iterable(df["tokens"].values.flatten())))
words.append("ENDPAD")
n_words = len(words)
word2idx = {w: i + 1 for i, w in enumerate(words)}
word2idx['international']

1931

In [10]:
def func(a):
    return [word2idx[i] for i in a]

In [11]:
#df.tokens.apply(func)

In [12]:
def reformatter(row):
    return [(row['tokens'][i], row['ner_tags'][i], row['pos_tags'][i]) 
            for i in range(row['sentence_length'])]

sentences = df[['ner_tags', 'pos_tags', 'tokens', 'sentence_length']].apply(reformatter, axis=1).to_numpy()

In [13]:
#X = df[['tokens']]; X

In [16]:
! pip3 install keras_contrib

ERROR: Could not find a version that satisfies the requirement keras_contrib (from versions: none)
ERROR: No matching distribution found for keras_contrib


In [15]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'keras_contrib'

In [ ]:
# Hyperparams if GPU is available
if tf.config.list_physical_devices('GPU'):
    BATCH_SIZE = 512  # Number of examples used in each iteration
    EPOCHS = 5  # Number of passes through entire dataset
    MAX_LEN = 75  # Max length of review (in words)
    EMBEDDING = 40  # Dimension of word embedding vector
# Hyperparams for CPU training
else:
    BATCH_SIZE = 32
    EPOCHS = 5
    MAX_LEN = 75
    EMBEDDING = 20

In [ ]:
df.head()

In [ ]:
tags = list(set(itertools.chain.from_iterable(df["ner_tags"].values.flatten())))
n_tags = len(tags)

In [ ]:
plt.hist(df['sentence_length'], bins=50);
plt.title('Token per sentence')
plt.xlabel('Len (number of token)')
plt.ylabel('# samples')
plt.show()

https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

In [ ]:
df.head()

https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

In [29]:
import spacy

In [30]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [31]:
ner = nlp.get_pipe("ner")

In [ ]:
# Adding labels to the `ner`

#for _, annotations in TRAIN_DATA:
    #for ent in annotations.get("entities"):
        #ner.add_label(ent[2])